# Data Pre Processing

This is a notebook for intaking a raw data file, preprocessing it, and exporting it in a format in which it can easily be labeled.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/directoryident.tsv", sep="\t")
df.head()

,dn,lastname,givenname,middle,dob,address
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,NaN
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,NaN,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,NaN,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,NaN,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,NaN,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...


## Preprocessing

### Cleanup

In [3]:
# Add unique ID column
# Uses the row number of original csv file as unique ID
df['Unique ID'] = df.index + 1

In [4]:
# Turn all NaN values into empty strings
df['address'] = df['address'].fillna(" ")
df['middle'] = df['middle'].fillna(" ")

In [5]:
df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,,1
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...,2
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...,3
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...,4
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...,5


In [6]:
# Convert all columns to string
df = df.astype(str)

In [7]:
# take sample of 5000
df = df.sample(n=500)

In [8]:
# Function to parse a singular dn field
# Returns the uid in the dn
def parse_dn(dn):
    dn = dn.split(',')
    dn = [x.split('=') for x in dn]
    dn = {x[0].strip(): x[1].strip() for x in dn}
    return dn["uid"]

In [9]:
# Function to concatenate every value in the address 
# We want to concatenate all values inside curly braces
def concat_address(address):
    if address != ' ':
        address = address.split('}:{')
        try: 
            address = [x.split('=')[1] for x in address]
        except:
            return "DELETE"
        address = ' '.join(address)
    return address

In [10]:
# Use vectorization to set new values for dn and address columns
df['dn'] = df['dn'].apply(parse_dn)
df['address'] = df['address'].apply(concat_address)
df['dob'] = df['dob'].apply(lambda x : x[:8])

In [11]:
if "DELETE" in df['address'].values:
    df = df[df['address'] != "DELETE"]

df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
537649,charityk,Kendall,C Catherine,,19570822,1202 Gardner Ave Ann Arbor MI 48104-4321 UNITE...,537650
233347,raburtch,Burtch,Rachel,,19960517,1935 Woodslea Dr Apt 11 Flint MI 48507-5240 UN...,233348
61396,manikana,Arnipalli,Manikanta Swamy,,19890715,"3815, 301B Greenbrier Ann Arbor MI 48105 UNITE...",61397
575109,stevegld,Goldstein,Steven,A,19540915,608 Green Rd Ann Arbor MI 48105 USA},575110
437877,csjean,Jean,Clark,Shihshiuan,19690712,3290 Saddle Soap Ct Las Vegas NV 89135-1838 UN...,437878


In [12]:
# Change column names
df.columns = ['ID', 'Last Name', 'First Name', 'Middle Name', 'Date of Birth', 'Address', 'Unique ID']
# Reorder columns
df = df[['Unique ID', 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date of Birth', 'Address']]

In [13]:
df.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
537649,537650,charityk,C Catherine,,Kendall,19570822,1202 Gardner Ave Ann Arbor MI 48104-4321 UNITE...
233347,233348,raburtch,Rachel,,Burtch,19960517,1935 Woodslea Dr Apt 11 Flint MI 48507-5240 UN...
61396,61397,manikana,Manikanta Swamy,,Arnipalli,19890715,"3815, 301B Greenbrier Ann Arbor MI 48105 UNITE..."
575109,575110,stevegld,Steven,A,Goldstein,19540915,608 Green Rd Ann Arbor MI 48105 USA}
437877,437878,csjean,Clark,Shihshiuan,Jean,19690712,3290 Saddle Soap Ct Las Vegas NV 89135-1838 UN...


### Similarity Measures

In [14]:
# Sample 50 records from df into new df
df_sample = df.sample(50)

In [15]:
df_sample.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
312590,312591,skua,Siang-Chean,,Kua,19930317,4604 Sterlington Dr Grovetown GA 30813-2277 UN...
282631,282632,cholts,Conrad,Ross,Holtsclaw,19941110,3360 Octavia St Apt 5 San Francisco CA 94123-2...
164764,164765,umrrhall,Randall,Ross,Hall,19530604,2610 Twin Eagles Dr Traverse City MI 49686-935...
747,748,tdevin,Devin,Christine,Tomlinson,19951210,
536010,536011,gablist,Gwenn,Sharon,Afton-Bird,19621106,


In [16]:
# Remove sampled records from df
df = df.drop(df_sample.index)

With a sample df, we can now build a similarity measure df by comparing each record in the sample df with the other records.

In [17]:
import Levenshtein as lev

In [18]:
"""
Returns normalized levenshtein distance between two strings
"""
def levenshtein_distance(
        s1: str,
        s2: str
) -> int:
    distance = lev.distance(s1, s2)
    try:
        return 1 - distance / float(max(len(s1), len(s2)))
    except ZeroDivisionError:
        return 0

In [19]:
"""
Returns Levenshtein distance between each field of two rows
Assumes following format for rows:
    row = pd.Series([First Name, Middle Name, Last Name, DOB, Address, ID])
"""
def row_similarity(
        row_1: pd.Series, 
        df_2: pd.DataFrame
    ) -> pd.DataFrame:

    # Compute similarity measures for each column using Levenshtein distance
    first_name_similarity = df_2['First Name'].apply(lambda x : levenshtein_distance(row_1['First Name'], x))
    middle_name_similarity = df_2['Middle Name'].apply(lambda x : levenshtein_distance(row_1['Middle Name'], x))
    last_name_similarity = df_2['Last Name'].apply(lambda x : levenshtein_distance(row_1['Last Name'], x))
    dob_similarity = df_2['Date of Birth'].apply(lambda x : levenshtein_distance(row_1['Date of Birth'], x))
    address_similarity = df_2['Address'].apply(lambda x : levenshtein_distance(row_1['Address'], x))
    id_similarity = df_2['ID'].apply(lambda x : levenshtein_distance(row_1['ID'], x))

    # Return DataFrame with similarity measures
    return pd.DataFrame({
        'UNIQ_ID1': row_1['Unique ID'],
        'UNIQ_ID2': df_2['Unique ID'],
        'ID1': row_1['ID'],
        'ID2': df_2['ID'],
        'ID Similarity': id_similarity,
        'First Name 1' : row_1['First Name'],
        'First Name 2' : df_2['First Name'],
        'First Name Similarity': first_name_similarity,
        'Middle Name 1' : row_1['Middle Name'],
        'Middle Name 2' : df_2['Middle Name'],
        'Middle Name Similarity': middle_name_similarity,
        'Last Name 1' : row_1['Last Name'],
        'Last Name 2' : df_2['Last Name'],
        'Last Name Similarity': last_name_similarity,
        'Date of Birth 1' : row_1['Date of Birth'],
        'Date of Birth 2' : df_2['Date of Birth'],
        'Date of Birth Similarity': dob_similarity,
        'Address 1' : row_1['Address'],
        'Address 2' : df_2['Address'],
        'Address Similarity': address_similarity
    })


In [20]:
"""
Builds similarity measure between records in two df
Creates a new df from the two df with the following columns:
    - ID1: ID of record in df1
    - ID2: ID of record in df2
    - First Name Similarity: Normalized levenshtein distance between first names
    - Middle Name Similarity: Normalized levenshtein distance between middle names
    - Last Name Similarity: Normalized levenshtein distance between last names
    - Date of Birth Similarity: Normalized levenshtein distance between dates of birth
    - Address Similarity: Normalized levenshtein distance between addresses
    - ID Similarity: Normalized levenshtein distance between IDs
"""
def build_similarity_df (
        df_1: pd.DataFrame,
        df_2: pd.DataFrame
) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=[ 'UNIQ_ID1',
                                    'UNIQ_ID2',
                                    'ID1', 
                                   'ID2', 
                                   'ID Similarity',
                                   'First Name 1',
                                   'First Name 2',
                                   'First Name Similarity', 
                                   'Middle Name 1',
                                    'Middle Name 2',
                                   'Middle Name Similarity',
                                    'Last Name 1',
                                    'Last Name 2', 
                                   'Last Name Similarity', 
                                    'Date of Birth 1',
                                    'Date of Birth 2',
                                   'Date of Birth Similarity', 
                                    'Address 1',
                                    'Address 2',
                                   'Address Similarity'])

    # Convert ID columns to string
    df_1["ID"] = df_1["ID"].astype(str)
    df_2["ID"] = df_2["ID"].astype(str)

    def apply_row_similarity(row, new_df):
        sim = row_similarity(row, df_2)
        new_df = pd.concat([new_df, sim], ignore_index=True)
        return new_df

    # Use vectorization to compute similarity between each row in df_1 and df_2
    new_df = df_1.apply(apply_row_similarity, args=(new_df,), axis=1).reset_index(drop=True)

    # New df is a series of dfs, so we need to concatenate them
    new_df = pd.concat(new_df.to_list(), ignore_index=True)

    return new_df

In [21]:
similarity_df = build_similarity_df(df_sample,  df)
similarity_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,312591,537650,skua,charityk,0.000000,Siang-Chean,C Catherine,0.090909,,,1.0,Kua,Kendall,0.285714,19930317,19570822,0.375,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,1202 Gardner Ave Ann Arbor MI 48104-4321 UNITE...,0.451613
1,312591,233348,skua,raburtch,0.125000,Siang-Chean,Rachel,0.272727,,,1.0,Kua,Burtch,0.166667,19930317,19960517,0.750,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,1935 Woodslea Dr Apt 11 Flint MI 48507-5240 UN...,0.419355
2,312591,61397,skua,manikana,0.250000,Siang-Chean,Manikanta Swamy,0.266667,,,1.0,Kua,Arnipalli,0.111111,19930317,19890715,0.500,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,"3815, 301B Greenbrier Ann Arbor MI 48105 UNITE...",0.387097
3,312591,575110,skua,stevegld,0.125000,Siang-Chean,Steven,0.272727,,A,0.0,Kua,Goldstein,0.000000,19930317,19540915,0.500,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,608 Green Rd Ann Arbor MI 48105 USA},0.290323
4,312591,418585,skua,hawklor,0.142857,Siang-Chean,Lori,0.000000,,M,0.0,Kua,Hawkins Hachenski,0.058824,19930317,19750601,0.375,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,18443 Country Club Ct Riverview MI 48193-8161 ...,0.453125


### Discretization

### WPKID

In [22]:
wpkid_df = similarity_df.copy()

In [23]:
# holds bin ranges for each similarity attribute
bin_ranges = {'First Name Similarity' : [],
        'Middle Name Similarity' : [],
        'Last Name Similarity' : [],
        'Date of Birth Similarity' : [],
        'Address Similarity' : [],
        'ID Similarity' : []}

In [24]:
#for solving systems of equations
import sympy as sp 
import math


t = # of intervals<br>
s = interval size<br>
n = # of instances in training dataset<br>

s * t = n<br>
s - 30 = t<br>


In [25]:
# WPKID function to solve for s and t given n, returns pair {s, t}
def interval_size_and_num(n) -> tuple:
    s, t = sp.symbols('s t')
    equation1 = sp.Eq(s * t, n)
    equation2 = sp.Eq(s - 30, t)  
    solution = sp.solve((equation1, equation2), (s, t))
    s_val = solution[0][1] * -1
    t_val = solution[1][1]

    s_int = math.floor(s_val)
    t_int = math.floor(t_val)
    return s_int, t_int

In [26]:
# function that takes in a column and discretizes it, saving bin ranges
def discretize_column(column, df):
    # separate zeros from nonzeros
    zeros_df = wpkid_df[wpkid_df[column] == 0].copy()
    nonzeros_df = wpkid_df[wpkid_df[column] != 0].copy()

    #get num_instances and num_intervals for nonzeros
    num_instances = interval_size_and_num(len(nonzeros_df))[0]
    num_intervals = interval_size_and_num(len(nonzeros_df))[1]

    # get bin ranges for nonzeros
    nonzeros_df[column], bin = pd.qcut(nonzeros_df[column], 
                                        q=num_intervals, 
                                        labels=False,
                                        retbins=True,
                                        duplicates='drop')
    
    #add 1 to every value to switch to index 1
    nonzeros_df[column] = nonzeros_df[column] + 1

    #save bin ranges
    bin_ranges[column].extend(bin.tolist())

    #recombine zero and nonzero dataframes
    df.loc[df[column] == 0, column] = zeros_df[column]
    df.loc[df[column] != 0, column] = nonzeros_df[column] 

    return df

In [27]:
wpkid_df = discretize_column('First Name Similarity', wpkid_df)
wpkid_df = discretize_column('Middle Name Similarity', wpkid_df)
wpkid_df = discretize_column('Last Name Similarity', wpkid_df)
wpkid_df = discretize_column('Date of Birth Similarity', wpkid_df)
wpkid_df = discretize_column('Address Similarity', wpkid_df)
wpkid_df = discretize_column('ID Similarity', wpkid_df)

In [28]:
# insert 0 at the beginning of each bin range
for key in bin_ranges:
    bin_ranges[key].insert(0, 0)

# if last value in bin range is not 1.0, add it
for key in bin_ranges:
    if bin_ranges[key][-1] != 1.0:
        bin_ranges[key].append(1.0)

In [29]:
"""
Export for prototype.py
"""
import pickle

# Create copy of bin ranges
# Turn bin ranges value into an integer representing size of the bin
with open('bin_ranges.pickle', 'wb') as handle:
    pickle.dump(bin_ranges, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
wpkid_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,312591,537650,skua,charityk,0.0,Siang-Chean,C Catherine,1.0,,,18.0,Kua,Kendall,16.0,19930317,19570822,2.0,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,1202 Gardner Ave Ann Arbor MI 48104-4321 UNITE...,66.0
1,312591,233348,skua,raburtch,1.0,Siang-Chean,Rachel,10.0,,,18.0,Kua,Burtch,9.0,19930317,19960517,5.0,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,1935 Woodslea Dr Apt 11 Flint MI 48507-5240 UN...,49.0
2,312591,61397,skua,manikana,4.0,Siang-Chean,Manikanta Swamy,10.0,,,18.0,Kua,Arnipalli,5.0,19930317,19890715,3.0,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,"3815, 301B Greenbrier Ann Arbor MI 48105 UNITE...",41.0
3,312591,575110,skua,stevegld,1.0,Siang-Chean,Steven,10.0,,A,0.0,Kua,Goldstein,0.0,19930317,19540915,3.0,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,608 Green Rd Ann Arbor MI 48105 USA},37.0
4,312591,418585,skua,hawklor,1.0,Siang-Chean,Lori,0.0,,M,0.0,Kua,Hawkins Hachenski,1.0,19930317,19750601,2.0,4604 Sterlington Dr Grovetown GA 30813-2277 UN...,18443 Country Club Ct Riverview MI 48193-8161 ...,67.0


In [31]:
for key, value in bin_ranges.items():
    print(key, value)

First Name Similarity [0, 0.05555555555555558, 0.09090909090909094, 0.11111111111111116, 0.125, 0.1428571428571429, 0.16666666666666663, 0.18181818181818177, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2857142857142857, 0.33333333333333337, 0.375, 0.4, 0.4285714285714286, 0.5, 0.6, 1.0]
Middle Name Similarity [0, 0.05882352941176472, 0.08333333333333337, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1428571428571429, 0.15384615384615385, 0.16666666666666663, 0.17765894236482446, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2727272727272727, 0.2857142857142857, 0.33333333333333337, 0.4, 0.5, 1.0]
Last Name Similarity [0, 0.05555555555555558, 0.05882352941176472, 0.06666666666666665, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.12358597285067925, 0.125, 0.1428571428571429, 0.16666666666666663, 0.18181818181818177, 0.1875, 0.19999999999999996, 0.2222222222222222, 0.25, 0.2727272727272727, 0.2857142857142857, 0.30000000000000004, 0.333

In [32]:
# Export wpkid_df to csv 
wpkid_df.to_csv('data/sampleunlabeledident.csv', index=False)